In [1]:
import cv2
import pandas as pd
import numpy as np
from matplotlib import pyplot, colors, gridspec
from ipywidgets import interact

import os
from os import path as osp

In [49]:
img_dir ='../../data/dataV0/'
data_root = '../../submission/mask2former_swin'
file_name = [f'fold-{i}' for i in range(4)]
#file_name.append('hard_voted_output')
len_file = len(file_name)

file = [pd.read_csv(osp.join(data_root, file_name[i] + '.csv')) for i in range(len_file)]
classes = [
    "Background",
    "General trash",
    "Paper",
    "Paper pack",
    "Metal",
    "Glass",
    "Plastic",
    "Styrofoam",
    "Plastic bag",
    "Battery",
    "Clothing",
]

PALLETE = [[  0,   0,   0], [128,   0,   0], [  0, 128,   0], [128, 128,   0],
           [  0,   0, 128], [128,   0, 128], [  0, 128, 128], [128, 128, 128],
           [ 64,   0,   0], [192,   0,   0],[ 64, 128,   0]]


In [58]:

def get_pred(kfold,i):
    row = file[kfold].iloc[i]
    img_id = row['image_id']
    
    
    pred = row['PredictionString']
    pred = np.array(pred.split(' '))
    pred = np.array(list(map(lambda x: PALLETE[int(x)], pred)))
    pred = pred.reshape((256,256,3))
    
    return pred

def show_img(i):
    row = file[0].iloc[i]
    preds = [get_pred(k, i) for k in range(len_file)]
    
    img_id = row['image_id']
    img = cv2.imread(osp.join(img_dir, img_id))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, [256, 256])
    

    fig = pyplot.figure(figsize=(20,10))
    gs_img = fig.add_gridspec(nrows=1, ncols = 1, left = 0, right = 0.4, top = 1, bottom=0.7)
    gs_color = fig.add_gridspec(nrows=1, ncols = 1, left = 0.4, right = 1, top = 1, bottom=0.7)
    gs_pred = fig.add_gridspec(nrows=2, ncols = len_file, left = 0, right = 1,top = 0.6, bottom=0)
    
    pyplot.subplot(gs_img[0,0])
    pyplot.title(img_id)
    pyplot.axis('off')
    pyplot.imshow(img)
    
    pyplot.subplot(gs_color[0,0])
    pyplot.barh(classes, [1 for _ in range(11)], color = [colors.to_hex([v /256 for v in color]) for color in PALLETE])
    pyplot.gca().axes.xaxis.set_visible(False)
    ax = pyplot.gca()
    for tick in ax.yaxis.get_major_ticks():
        tick.label.set_fontsize(20)
    #pyplot.show()
    
    #print just pred
    for file_num in range(len_file):
        pyplot.subplot(gs_pred[0,file_num])
        pyplot.axis('off')
        pyplot.title(file_name[file_num])
        pyplot.imshow(preds[file_num])

        pyplot.subplot(gs_pred[1,file_num])
        pyplot.axis('off')
        pyplot.title(file_name[file_num] + 'Mix')
        pyplot.imshow(img)
        pyplot.imshow(preds[file_num], alpha=0.5)

interact(show_img, i=range(100))

interactive(children=(Dropdown(description='i', options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15,…

<function __main__.show_img(i)>